# Food Delivery Data Integration

This notebook combines three different data files to create the final food delivery dataset:
1. **orders.csv** - Transactional order data
2. **users.json** - User master data
3. **restaurants.sql** - Restaurant master data

## Steps:
1. Load CSV Data (orders)
2. Load JSON Data (users)
3. Load SQL Data (restaurants)
4. Merge the Data
5. Create Final Dataset

In [ ]:
# Import required libraries
import pandas as pd
import json
import re
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

## Step 1: Load CSV Data (Orders)

In [ ]:
# Load the orders CSV file
orders_df = pd.read_csv('orders.csv')

print("Orders Dataset:")
print(f"Shape: {orders_df.shape}")
print(f"\nColumns: {orders_df.columns.tolist()}")
print(f"\nData Types:\n{orders_df.dtypes}")
orders_df.head()

In [ ]:
# Check for missing values in orders
print("Missing Values in Orders:")
print(orders_df.isnull().sum())

## Step 2: Load JSON Data (Users)

In [ ]:
# Load the users JSON file
with open('users.json', 'r') as f:
    users_data = json.load(f)

# Convert to DataFrame
users_df = pd.DataFrame(users_data)

print("Users Dataset:")
print(f"Shape: {users_df.shape}")
print(f"\nColumns: {users_df.columns.tolist()}")
print(f"\nData Types:\n{users_df.dtypes}")
users_df.head()

In [ ]:
# Check for missing values in users
print("Missing Values in Users:")
print(users_df.isnull().sum())

In [ ]:
# Quick summary of user data
print("User Cities Distribution:")
print(users_df['city'].value_counts())
print("\nMembership Distribution:")
print(users_df['membership'].value_counts())

## Step 3: Load SQL Data (Restaurants)

In [ ]:
# Function to parse SQL INSERT statements
def parse_sql_file(filename):
    """
    Parse SQL file containing INSERT statements and extract data.
    """
    restaurants = []
    
    with open(filename, 'r') as f:
        content = f.read()
    
    # Regular expression to match INSERT statements
    # Pattern: INSERT INTO restaurants VALUES (id, 'name', 'cuisine', rating);
    pattern = r"INSERT INTO restaurants VALUES \((\d+),\s*'([^']+)',\s*'([^']+)',\s*([\d.]+)\)"
    
    matches = re.findall(pattern, content)
    
    for match in matches:
        restaurant_id = int(match[0])
        restaurant_name = match[1]
        cuisine = match[2]
        rating = float(match[3])
        
        restaurants.append({
            'restaurant_id': restaurant_id,
            'restaurant_name_sql': restaurant_name,
            'cuisine': cuisine,
            'rating': rating
        })
    
    return pd.DataFrame(restaurants)

# Load restaurants from SQL file
restaurants_df = parse_sql_file('restaurants.sql')

print("Restaurants Dataset:")
print(f"Shape: {restaurants_df.shape}")
print(f"\nColumns: {restaurants_df.columns.tolist()}")
print(f"\nData Types:\n{restaurants_df.dtypes}")
restaurants_df.head()

In [ ]:
# Quick summary of restaurant data
print("Cuisine Distribution:")
print(restaurants_df['cuisine'].value_counts())
print(f"\nRating Statistics:")
print(restaurants_df['rating'].describe())

## Step 4: Merge the Data

Perform joins using keys:
- `orders.user_id` → `users.user_id`
- `orders.restaurant_id` → `restaurants.restaurant_id`

Join Type: **Left Join** (to retain all orders)

In [ ]:
# First, merge orders with users on user_id
merged_df = orders_df.merge(
    users_df,
    on='user_id',
    how='left'
)

print(f"After merging with users:")
print(f"Shape: {merged_df.shape}")
print(f"Columns: {merged_df.columns.tolist()}")
merged_df.head()

In [ ]:
# Now, merge with restaurants on restaurant_id
final_df = merged_df.merge(
    restaurants_df,
    on='restaurant_id',
    how='left'
)

print(f"After merging with restaurants:")
print(f"Shape: {final_df.shape}")
print(f"Columns: {final_df.columns.tolist()}")
final_df.head()

In [ ]:
# Check for any unmatched records
print("Checking for unmatched records:")
print(f"Users not matched: {final_df['name'].isnull().sum()}")
print(f"Restaurants not matched: {final_df['cuisine'].isnull().sum()}")

## Step 5: Create Final Dataset

The final dataset contains:
- Order details
- User information
- Restaurant information

In [ ]:
# Rename columns for clarity
final_df = final_df.rename(columns={
    'name': 'user_name',
    'city': 'user_city'
})

# Display final column list
print("Final Dataset Columns:")
for col in final_df.columns:
    print(f"  - {col}")

In [ ]:
# View final dataset info
print("Final Dataset Info:")
print(f"Total Records: {len(final_df)}")
print(f"Total Columns: {len(final_df.columns)}")
print("\nData Types:")
print(final_df.dtypes)

In [ ]:
# Display first few rows of final dataset
print("Final Dataset Preview:")
final_df.head(10)

In [ ]:
# Statistical summary
print("Statistical Summary:")
final_df.describe()

In [ ]:
# Check for missing values in final dataset
print("Missing Values in Final Dataset:")
print(final_df.isnull().sum())

In [ ]:
# Save the final dataset to CSV
output_filename = 'final_food_delivery_dataset.csv'
final_df.to_csv(output_filename, index=False)

print(f"\n✅ Final dataset saved successfully as '{output_filename}'")
print(f"   - Total records: {len(final_df)}")
print(f"   - Total columns: {len(final_df.columns)}")

## Summary

The final dataset has been created by combining:
1. **Orders** (CSV) - Transactional data
2. **Users** (JSON) - User master data
3. **Restaurants** (SQL) - Restaurant master data

Using **Left Joins** to retain all orders and adding user/restaurant information.

In [ ]:
# Quick verification - read back the saved file
verification_df = pd.read_csv('final_food_delivery_dataset.csv')
print(f"Verification: File saved with {len(verification_df)} records and {len(verification_df.columns)} columns")
verification_df.head()